In [1]:
from datasets import FirstTokenRepeatedLast
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = FirstTokenRepeatedLast(num_points=1000)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[17,  1,  9,  7, 12,  5,  6,  8, 11, 17],
         [18,  4, 18, 18,  5, 19, 18,  3,  4, 19],
         [ 2,  7, 13, 16, 17, 19, 14,  2,  4,  2],
         [ 8, 13, 13, 14, 19, 12, 12, 11,  7, 17],
         [19, 10, 12, 17, 10,  1,  5, 12,  4, 19],
         [17,  3,  3,  6, 17,  7, 14, 18,  6, 13],
         [ 8,  3, 16, 15, 19, 17, 12,  2, 11,  8],
         [ 9,  4, 12,  8, 10, 14,  7,  8,  9,  9],
         [19, 12,  2,  2,  7,  7,  6,  5,  3, 19],
         [15, 14,  4,  2, 19, 15,  3, 12, 12,  3]]),
 tensor([1, 0, 1, 0, 1, 0, 1, 1, 1, 0]))

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 0    354
1    346
dtype: int64
Val data:
 1    59
0    41
dtype: int64
Test data:
 0    105
1     95
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-03-24 02:08:16.012572: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [6]:
np.log(2)

0.6931471805599453

In [7]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6931898593902588, grad: 0.0),
 node(name: +, val: [[-8.93743359e-04 -5.39658475e-04]
  [-6.30752358e-04 -1.24714256e-03]
  [-7.08709704e-04  3.45047534e-04]
  [ 3.19276674e-04  6.54158939e-04]
  [-2.30401944e-04  7.82586168e-04]
  [-7.08558015e-04  3.45299341e-04]
  [ 5.65697381e-04 -7.55890460e-06]
  [ 3.18721432e-04  6.53633615e-04]
  [-1.05148694e-03 -5.74758509e-04]
  [-3.56030738e-04 -1.33626818e-04]
  [ 5.64655871e-04 -8.39027325e-06]
  [-1.52113003e-04 -6.14569406e-04]
  [-8.93747841e-04 -5.39616332e-04]
  [-2.84601439e-04 -6.64322462e-04]
  [-3.55833356e-04 -1.33451234e-04]
  [ 7.18896103e-04  5.09075762e-04]
  [ 7.18688592e-04  5.09047299e-04]
  [-3.20820109e-04  5.66291157e-04]
  [-2.84989219e-04 -6.64601102e-04]
  [ 3.18753358e-04  6.53514348e-04]
  [ 1.77451002e-04  5.84069057e-04]
  [ 6.41890627e-04  1.27545313e-03]
  [-3.55731143e-04 -1.33368565e-04]
  [ 3.19092418e-04  6.53770694e-04]
  [-1.51831016e-04 -6.14517950e-04]
  [ 2.30242676e-05 -3.4557058